In [18]:
import os, json, requests, datetime
import pandas as pd

geo_info = pd.read_csv("./geo_info.csv")
origin = f'{geo_info.iloc[0]["Latitude"]},{geo_info.iloc[0]["Longitude"]}'
destination = f'{geo_info.iloc[1]["Latitude"]},{geo_info.iloc[1]["Longitude"]}'

print(origin)
print(destination)

# 네이버 지도 API 키 설정
SECRET_PATH = "./.secret.json"

client_id = None
client_secret = None

with open(SECRET_PATH) as f:
    secrets = json.load(f)
    client_id = secrets["CLIENT_ID"]
    client_secret = secrets["CLIENT_SECRET"]


def get_travel_info(origin, destination, departure_time, mode):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving" if mode == "driving" else "https://naveropenapi.apigw.ntruss.com/map-direction/v1/walking" if mode == "walking" else "https://naveropenapi.apigw.ntruss.com/map-direction/v1/transit"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    params = {
        "start": origin,
        "goal": destination,
        "option": "traoptimal" if mode == "driving" else "fast" if mode == "transit" else "shortest",
        "departureTime": departure_time
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        result = response.json()
        if mode == "driving":
            print(result)
            travel_time = result['route']['traoptimal'][0]['summary']['duration']  # milliseconds
            cost = result['route']['traoptimal'][0]['summary'].get('tollFare', 0) * 1.2  # 예상 택시 요금 (대략 1.2배)
        elif mode == "walking":
            travel_time = result['route']['traoptimal'][0]['summary']['duration']  # milliseconds
            cost = 0
        elif mode == "transit":
            travel_time = result['route']['traoptimal'][0]['summary']['duration']  # milliseconds
            cost = result['route']['traoptimal'][0]['summary']['fare']
        return travel_time / 1000, cost  # seconds, cost
    else:
        print("Error:", response.status_code)
        return None, None

# 예시: 특정 시간에 A에서 B로 이동하는 데 걸리는 시간 및 비용 계산
departure_time = int(datetime.datetime.now().timestamp()) * 1000  # milliseconds
travel_time, cost = get_travel_info(origin, destination, departure_time, mode="transit")
print(f"Travel time: {travel_time / 60} minutes, Cost: {cost} KRW")


37.5511694,126.9882266
37.579617,126.977041
Error: 404


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'